### Licença

Este Jupyter Notebook foi adaptado a partir do [tutorial oficial da Hopsworks](https://github.com/logicalclocks/hopsworks-tutorials/tree/master/advanced_tutorials/citibike), que está sob a [GNU Affero General Public License v3.0 (AGPLv3)](https://www.gnu.org/licenses/agpl-3.0.html). Em conformidade com esta licença, este trabalho e todas as obras derivadas devem também ser compartilhadas sob os mesmos termos.


# Previsão de Uso do Citibike

<small>01. Feature Backfill</small>

<center>
<img src='../../assets/m08_projeto_banner.png' alt='Penguins' width="600">
</center>

🗒️ O objetivo deste projeto é demonstrar criar uma solução real de *Machine Learning* para prever o número de usuários da Citi Bike em cada estação na cidade de Nova York. Especificamente, você irá criar uma *Feature Store* com o Hopsworks, e fazer o *deploy* de uma aplicação com o Streamlit. Este projeto está dividido nas seguintes partes:

- **Backfill de Features**: Como carregar, engenhar e criar grupos de features.
- **Pipeline de Features**: Como analisar novos dados e inseri-los nos grupos de features.
- **Pipeline de Treinamento**: Como construir uma visualização de features, divisão do conjunto de dados de treinamento, treinar um modelo e salvá-lo no Registro de Modelos.
- **Pipeline de Inferência**: Como recuperar um modelo treinado do registro de modelos e usá-lo para inferência em lote.
- **Implantar um aplicativo Streamlit**.

# Previsão de Uso da Citibike

Este é um exemplo avançado do uso do Hopsworks Feature Store, com a tarefa de prever o número de usuários da Citibike em cada estação na cidade de Nova York. O Feature Store é uma parte essencial da infraestrutura de IA que ajuda as organizações a trazer dados empresariais modernos para sistemas de ML analíticos e operacionais. É a maneira mais simples e poderosa de levar seus modelos para a produção, de qualquer lugar para qualquer lugar. Você carregará os dados iniciais no feature store, criará dois grupos de features a partir dos quais faremos uma visualização de features e um conjunto de dados de treinamento, e treinará um modelo para prever a quantidade de usuários. Além disso, você projetará um pipeline de geração de dados e inserção no Feature Store, que será executado periodicamente usando ações do GitHub. Um aplicativo Streamlit será criado para que você possa experimentar seu modelo de maneira interativa.

Este é um caso de uso em lote, que oferecerá uma visão de alto nível sobre como usar nossas APIs Python e a interface do usuário para navegar pelos grupos de features.


In [5]:
# Carregar variáveis de ambiente
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv

# Bibliotecas padrão
from datetime import timedelta, datetime
import os

# Bibliotecas de análise de dados
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar

# Bibliotecas de visualização
import plotly.express as px

# Módulos específicos do projeto
import sigmoidal as sig

# Configurações gerais
import warnings
warnings.filterwarnings("ignore")  # Ignorar avisos


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [6]:
HOPSWORKS_API_KEY = os.getenv("HOPSWORKS_API_KEY")
HOPSWORKS_PROJECT_NAME = 'sigmoidal'

In [7]:
df_raw = sig.get_citibike_data("01/2023", "10/2023")

_____ Processando 01/2023... _____
https://s3.amazonaws.com/tripdata/202301-citibike-tripdata.csv.zip
Recuperando DataFrame do arquivo csv existente...💿
_____ Processando 02/2023... _____
https://s3.amazonaws.com/tripdata/202302-citibike-tripdata.csv.zip
Recuperando DataFrame do arquivo csv existente...💿
_____ Processando 03/2023... _____
https://s3.amazonaws.com/tripdata/202303-citibike-tripdata.csv.zip
Recuperando DataFrame do arquivo csv existente...💿
_____ Processando 04/2023... _____
https://s3.amazonaws.com/tripdata/202304-citibike-tripdata.csv.zip
Recuperando DataFrame do arquivo csv existente...💿
_____ Processando 05/2023... _____
https://s3.amazonaws.com/tripdata/202305-citibike-tripdata.csv.zip
Recuperando DataFrame do arquivo csv existente...💿
_____ Processando 06/2023... _____
https://s3.amazonaws.com/tripdata/202306-citibike-tripdata.csv.zip
Recuperando DataFrame do arquivo csv existente...💿
_____ Processando 07/2023... _____
https://s3.amazonaws.com/tripdata/202307-citibi

In [8]:
df_raw

,date,station_id,users_count
0,2023-10-01,3633.08,37
1,2023-10-01,3731.11,21
2,2023-10-01,3736.03,15
3,2023-10-01,3803.09,27
4,2023-10-01,3834.10,14
...,...,...,...
666505,2023-01-31,8226.06,8
666506,2023-01-31,8226.07,5
666507,2023-01-31,8336.02,6
666508,2023-01-31,8511.08,1


In [9]:
# converter coluna para string
df_raw['station_id'] = df_raw.station_id.astype(str)

In [10]:
# existem linhas duplicadas (vários registros para o mesmo dia e estação). vamos removê-las.
df_res = df_raw.copy()
df_res = df_res.groupby(['date', 'station_id'], as_index=False)['users_count'].sum()

df_res['prev_users_count'] = df_res.groupby('station_id')[
    'users_count'
].shift(+1)
df_res = df_res.dropna()
df_res = sig.moving_average(df_res, 7)
df_res = sig.moving_average(df_res, 14)

for i in [7, 14]:
    for func in [
        sig.moving_std,
        sig.exponential_moving_average,
        sig.exponential_moving_std,
    ]:
        df_res = func(df_res, i)
df_res = df_res.reset_index(drop=True)
df_res = df_res.sort_values(by=['date', 'station_id']).dropna()

df_res.head()

,date,station_id,users_count,prev_users_count,mean_7_days,mean_14_days,std_7_days,exp_mean_7_days,exp_std_7_days,std_14_days,exp_mean_14_days,exp_std_14_days
0,2023-01-02,2932.03,12,3.0,7.142857,6.071429,5.367450,6.506504,5.057768,4.393427,6.286295,4.570934
1,2023-01-02,2951.05,4,2.0,12.571429,12.285714,7.634508,10.978602,6.856891,5.483241,11.905087,6.401791
2,2023-01-02,3117.05,4,10.0,11.571429,10.214286,4.157609,12.026337,4.462319,4.172779,10.922464,4.265283
3,2023-01-02,3125.09,10,11.0,13.571429,12.500000,4.197505,12.387136,3.553464,4.146361,12.617410,4.165330
4,2023-01-02,3166.03,6,3.0,7.428571,7.142857,2.992053,7.588742,3.088086,2.957575,7.754732,3.401655


In [ ]:
# check if there are any null values
df_res.isna().sum()

In [ ]:
# describe the dataset
df_res.info()

In [ ]:
# pick a random station, using method sample, and filter the dataset
station_id = df_res.station_id.sample().values[0]
df_res[df_res.station_id == station_id]